<a href="https://colab.research.google.com/github/atotev/ca683-2021-dara/blob/main/Importing_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing


In [4]:
apiKey = "defc104635dae32057dd64f95aaf4e0d"


In [5]:
link ='https://drive.google.com/file/d/1rIJjrQh3g6QScqx6J-rN2kC8Ki2weP9v/view?usp=sharing'

In [6]:


downloaded = drive.CreateFile({'id':'1rIJjrQh3g6QScqx6J-rN2kC8Ki2weP9v'}) 
fileMovies = 'movie_titles.csv'

downloaded.GetContentFile(fileMovies)  

movies_netflix = pd.read_csv(fileMovies, 
                           encoding = 'ISO-8859-1', 
                           header = None, 
                           names = ['id', 'year', 'title']).set_index('id')

print('Shape Movie-Titles:\t{}'.format(movies_netflix.shape))
movies_netflix.sample(5)

Shape Movie-Titles:	(17770, 2)


,year,title
id,,
12479,2002.0,Marriage Is a Crazy Thing
4092,1997.0,How To Be A Player
16078,1967.0,Dark Shadows: Vol. 4
12048,1999.0,Timegate: Tales of the Saddle Tramps
6066,2004.0,The Story of the Weeping Camel


In [7]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Search for movies in the Movie DB API**
- For each movie found store genre, popularity, votes, genre and media type
- Ignore titles found with category 'People' as it would actors/directores an dnov movies

https://developers.themoviedb.org/3/search/search-movies

In [8]:
def getFirstPartMovieName(movie):
  if movie.find(":") != -1:
    return movie[0:movie.find(":")]
  if movie.find("(") != -1:
    return movie[0:movie.find("(")]
  if movie.find("/") != -1:
    return movie[0:movie.find("/")]
  return movie

def period_is_ok(release_date, year):
    if (len(release_date)>=4 and len(year) >= 4):
      release_date_int = int(float(release_date[0:4]))
      year_int = int(float(year[0:4]))
      difference = (release_date_int - year_int)
      if (difference >= -3 and difference <= 3 ):
        return True
    return False

In [9]:
#add new columns to the data frame
if 'id_moviedb' not in movies_netflix:
  movies_netflix['id_moviedb'] = 0
if 'genre_ids' not in movies_netflix:
  movies_netflix['genre_ids'] = str()
if 'vote_average' not in movies_netflix:
  movies_netflix['vote_average'] = 0
if 'genre_ids' not in movies_netflix:
  movies_netflix['genre_ids'] = 0
if 'media_type' not in movies_netflix:
  movies_netflix['media_type'] = ""
if 'popularity' not in movies_netflix:
  movies_netflix['popularity'] = ""

In [52]:
from google.colab import files
import requests
from time import sleep
count = 0

for ind in movies_netflix.index: 

    movie_title = movies_netflix['title'][ind]
    year = (str(movies_netflix['year'][ind])[0:4])

    movie_title_part = getFirstPartMovieName(movie_title)

    url = "https://api.themoviedb.org/3/search/multi?api_key="+ apiKey +"&language=en-US&query=" + str(movie_title_part)
    response = requests.get(url)
    results = response.json()

    movie = {}

    if (len(results["results"])) >= 1:
      for item in results["results"]:
        if item["media_type"] != "person":
          if "name" in item:            
            if item["name"] == movie_title:
              movie = item
              break
          if "title" in item: 
            if item["title"] == movie_title:
              movie = item
              break
          if "original_title" in item:             
            if item["original_title"] == movie_title:
              movie = item
              break
          if "release_date" in item:                   
            if period_is_ok(item["release_date"], year):
              movie = item
              break
          if "first_air_date" in item:    
            if period_is_ok(item["first_air_date"], year):
              movie = item
              break
             
    elif (len(results["results"])) == 1:
      movie = results["results"][0]
        
    if movie:
      count = count + 1
      movies_netflix['id_moviedb'][ind] = movie["id"]
      if "genre_ids" in movie:
        movies_netflix['genre_ids'][ind] = movie["genre_ids"]
      if "vote_average" in movie:
        movies_netflix['vote_average'][ind] = movie["vote_average"]
      if "media_type" in movie:
        movies_netflix['media_type'][ind] = movie["media_type"]
      if "popularity" in movie:
        movies_netflix['popularity'][ind] = movie["popularity"]

    if count == 10:
      # movies_netflix.to_csv('movie_titles.csv', header=True)
      # !cp movie_titles.csv "gdrive/My Drive/"
      count = 0
      sleep(1.5) #due to the limit of calls of the API


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to b

**Get List of Genre from API**

In [42]:
url = "https://api.themoviedb.org/3/genre/movie/list?api_key="+ apiKey +"&language=en-US"
response = requests.get(url)
results = response.json()
genres_list = results["genres"]
print(genres_list)

[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 99, 'name': 'Documentary'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 14, 'name': 'Fantasy'}, {'id': 36, 'name': 'History'}, {'id': 27, 'name': 'Horror'}, {'id': 10402, 'name': 'Music'}, {'id': 9648, 'name': 'Mystery'}, {'id': 10749, 'name': 'Romance'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 10770, 'name': 'TV Movie'}, {'id': 53, 'name': 'Thriller'}, {'id': 10752, 'name': 'War'}, {'id': 37, 'name': 'Western'}]


**One-hot enconding**

In [60]:
for ind in movies_netflix.index: 
  for genre in movies_netflix['genre_ids'][ind]:
    genre_found = [x for x in genres_list if x["id"] == int(genre)]
    if genre_found:
      if genre_found[0]["name"] not in movies_netflix:
        movies_netflix[genre_found[0]["name"]] = 0
      movies_netflix[genre_found[0]["name"]][ind] = 1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [61]:
movies_netflix.head(35)

,year,title,id_moviedb,genre_ids,vote_average,media_type,popularity,Adventure,Science Fiction,Documentary,Horror,Action,Animation,Comedy,Thriller,Mystery,Romance,Fantasy,Family,Crime,Drama,Music,History,TV Movie,War,Western
id,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2003.0,Dinosaur Planet,58691,"[12, 878]",3,movie,4.931,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2004.0,Isle of Man TT 2004 Review,0,,0,,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1997.0,Character,655068,[99],0,movie,0.600,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1994.0,Paula Abdul's Get Up & Dance,274766,[],0,movie,0.600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2004.0,The Rise and Fall of ECW,33209,[99],8,movie,7.537,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1997.0,Sick,308867,"[878, 27]",5,movie,5.666,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1992.0,8 Man,666282,"[28, 16, 878]",0,movie,0.600,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2004.0,What the #$*! Do We Know!?,8357,[99],5,movie,9.989,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1991.0,Class of Nuke 'Em High 2,26230,"[35, 27, 878]",4,movie,7.440,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


**Pre-Processing**

In [59]:
le = preprocessing.LabelEncoder()
# movies_netflix["media_type"] = le.fit_transform(movies_netflix["media_type"].astype(str))
# np.unique(movies_netflix["media_type"])
movies_netflix['popularity'] = pd.to_numeric(movies_netflix['popularity'], errors='coerce')
movies_netflix.dtypes

year               float64
title               object
id_moviedb           int64
genre_ids           object
vote_average         int64
media_type          object
popularity         float64
Adventure            int64
Science Fiction      int64
Documentary          int64
Horror               int64
Action               int64
Animation            int64
Comedy               int64
Thriller             int64
Mystery              int64
Romance              int64
Fantasy              int64
Family               int64
Crime                int64
Drama                int64
Music                int64
History              int64
TV Movie             int64
War                  int64
Western              int64
dtype: object